##### Copyright 2018 The TensorFlow Authors.
*https://www.tensorflow.org/tutorials/text/image_captioning

#### Traducido para Samsung DesArrolladoras.


In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Image captioning con atención visual


Dada una imagen como la del ejemplo a continuación, nuestro objetivo es generar una leyenda como "un surfista montando en una ola".

![Man Surfing](https://tensorflow.org/images/surf.jpg)

*[Image Source](https://commons.wikimedia.org/wiki/Surfing#/media/File:Surfing_in_Hawaii.jpg); License: Public Domain*

Para lograr esto, utilizamos un modelo basado en la atención, que nos permite ver en qué partes de la imagen se enfoca el modelo a medida que genera un título.

![Prediction](https://tensorflow.org/images/imcap_prediction.png)

El modelo de arquitectura es similar al del artículo: [Show, Attend and Tell: Neural Image Caption Generation with Visual Attention](https://arxiv.org/abs/1502.03044).

Este cuaderno es un ejemplo de principio a fin. Cuando ejecutas el cuaderno, descarga el dataset [MS-COCO](http://cocodataset.org/#home) , preprocesas y almacenas en caché un subconjunto de imágenes usando Inception V3, entrenas un modelo codificador-decodificador y generas subtítulos en nuevas imágenes usando el modelo entrenado.

En este ejemplo, entrenarás un modelo en una cantidad relativamente pequeña de datos: los primeros 30,000 subtítulos para aproximadamente 20,000 imágenes (ya que hay múltiples subtítulos por imagen en el conjunto de datos).


In [0]:
import tensorflow as tf

# Generarás plots de atención para ver en qué partes de una imagen
# se enfoca nuestro modelo durante el proceso de los subtítulos
import matplotlib.pyplot as plt

# Scikit-learn incluye muchas herramientas útiles
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import re
import numpy as np
import os
import time
import json
from glob import glob
from PIL import Image
import pickle

## Descarga y preparación del dataset MS-COCO 

Usarás [MS-COCO dataset](http://cocodataset.org/#home) para entrenar al modelo. El conjunto de datos contiene más de 82,000 imágenes, cada una de las cuales tiene al menos 5 anotaciones de subtítulos diferentes. El siguiente código descarga y extrae el conjunto de datos automáticamente.

**Cuidado: descarga de archivo muy grande**. Usarás el conjunto de entrenamiento, que es un archivo de 13GB.


In [0]:
# Descargar archivos de anotaciones de subtítulos
annotation_folder = '/annotations/'
if not os.path.exists(os.path.abspath('.') + annotation_folder):
  annotation_zip = tf.keras.utils.get_file('captions.zip',
                                          cache_subdir=os.path.abspath('.'),
                                          origin = 'http://images.cocodataset.org/annotations/annotations_trainval2014.zip',
                                          extract = True)
  annotation_file = os.path.dirname(annotation_zip)+'/annotations/captions_train2014.json'
  os.remove(annotation_zip)

# Descarga archivo de imágenes
image_folder = '/train2014/'
if not os.path.exists(os.path.abspath('.') + image_folder):
  image_zip = tf.keras.utils.get_file('train2014.zip',
                                      cache_subdir=os.path.abspath('.'),
                                      origin = 'http://images.cocodataset.org/zips/train2014.zip',
                                      extract = True)
  PATH = os.path.dirname(image_zip) + image_folder
  os.remove(image_zip)
else:
  PATH = os.path.abspath('.') + image_folder

## Opcional: limitar el tamaño del conjunto de entrenamiento
Para acelerar el entrenamiento para este tutorial, usarás un subconjunto de 30,000 subtítulos y sus imágenes correspondientes para entrenar a nuestro modelo. Elegir usar más datos daría como resultado una mejor calidad de subtítulos.

In [0]:
# Leemos el archivo json 
with open(annotation_file, 'r') as f:
    annotations = json.load(f)

# Almacenamos los subtítulos y los nombres de las imágenes en vectores 
all_captions = []
all_img_name_vector = []

for annot in annotations['annotations']:
    caption = '<start> ' + annot['caption'] + ' <end>'
    image_id = annot['image_id']
    full_coco_image_path = PATH + 'COCO_train2014_' + '%012d.jpg' % (image_id)

    all_img_name_vector.append(full_coco_image_path)
    all_captions.append(caption)

# Mezclamos los subtítulos y los nombres de imagen juntos
# Establecemos un estado aleatorio
train_captions, img_name_vector = shuffle(all_captions,
                                          all_img_name_vector,
                                          random_state=1)

# Seleccionamos los primeros 30000 subtítulos del conjunto aleatorio
num_examples = 30000
train_captions = train_captions[:num_examples]
img_name_vector = img_name_vector[:num_examples]

In [0]:
len(train_captions), len(all_captions)

## Preprocesar las imágenes empleando el InceptionV3

A continuación, utilizarás InceptionV3 (que está entrenado previamente en Imagenet) para clasificar cada imagen. Extraerás entidades de la última capa convolucional.

Primero, convertirás las imágenes al formato esperado de InceptionV3:

* Cambia el tamaño de la imagen a 299px by 299px
* [Preprocesa las imágenes](https://cloud.google.com/tpu/docs/inception-v3-advanced#preprocessing_stage) usando el [preprocess_input](https://www.tensorflow.org/api_docs/python/tf/keras/applications/inception_v3/preprocess_input) método para normalizar la imagen de modo que contenga píxeles en el rango de -1 a 1, que coincide con el formato de las imágenes utilizadas para entrenar a InceptionV3.


In [0]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

## Inicializa InceptionV3 y carga los pesos de Imagenet previamente entrenados.


Ahora crearás un modelo tf.keras donde la capa de salida es la última capa convolucional en la arquitectura InceptionV3. La forma de la salida de esta capa es '' 8 x 8 x 2048 ''. Emplea la última capa convolucional porque está utilizando la atención en este ejemplo. No realiza esta inicialización durante el entrenamiento porque podría convertirse en un cuello de botella.

* Reenvíarás cada imagen a través de la red y almacenarás el vector resultante en un diccionario (nombre_imagen -> elemento_vector).
* Después de que todas las imágenes pasen a través de la red, seleccionas el diccionario y lo guardas en el disco.


In [0]:
image_model = tf.keras.applications.InceptionV3(include_top=False,
                                                weights='imagenet')
new_input = image_model.input
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

## Almacenamiento en caché de las características extraídas de InceptionV3

Preprocesarás cada imagen con InceptionV3 y almacenarás en caché la salida en el disco. El almacenamiento en caché de la salida en RAM sería más rápido, pero también requeriría mucha memoria, y requeriría 8 \ * 8 \ * 2048 floats por imagen. Al momento de escribir este notebook, esto excede las limitaciones de memoria de Colab (actualmente 12GB de memoria).

El rendimiento podría mejorarse con una estrategia de almacenamiento en caché más sofisticada (por ejemplo, fragmentando las imágenes para reducir la I/O del disco de acceso aleatorio), pero eso requeriría más código.

El almacenamiento en caché tardará unos 10 minutos en ejecutarse en Colab con una GPU. Si deseas ver una barra de progreso, puedes:

1. instalar [tqdm](https://github.com/tqdm/tqdm):

    `!pip install -q tqdm`

2. Importar tqdm:

    `from tqdm import tqdm`

3. Cambiar la siguiente línea de código:

    `for img, path in image_dataset:`

    to:

    `for img, path in tqdm(image_dataset):`


In [0]:
# Obtenemos imágenes únicas

encode_train = sorted(set(img_name_vector))

# Siéntete libre de cambiar el batch_size según la configuración de tu sistema
image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
image_dataset = image_dataset.map(
  load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(16)

for img, path in image_dataset:
  batch_features = image_features_extract_model(img)
  batch_features = tf.reshape(batch_features,
                              (batch_features.shape[0], -1, batch_features.shape[3]))

  for bf, p in zip(batch_features, path):
    path_of_feature = p.numpy().decode("utf-8")
    np.save(path_of_feature, bf.numpy())

## Preprocesar y tokenizar los subtítulos (captions)

* Primero, tokenizarás los subtítulos (por ejemplo, dividiendo en espacios). Esto nos da un vocabulario de todas las palabras únicas en los datos (por ejemplo, "surf", "fútbol", etc.).
* A continuación, limitarás el tamaño del vocabulario a las 5,000 palabras principales (para ahorrar memoria). Reemplazarás todas las demás palabras con el token "UNK" (desconocido).
* Luego creas asignaciones de 'palabra a índice' e 'índice a palabra'.
* Y finalmente, rellenas todas las secuencias para que tengan la misma longitud que la más larga.

In [0]:
# Encuentra la longitud máxima de cualquier título en nuestro conjunto de datos
def calc_max_length(tensor):
    return max(len(t) for t in tensor)

In [0]:
# Elige las 5000 palabras principales del vocabulario
top_k = 5000
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                  oov_token="<unk>",
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(train_captions)
train_seqs = tokenizer.texts_to_sequences(train_captions)

In [0]:
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'

In [0]:
# Crea los vectores tokenizados
train_seqs = tokenizer.texts_to_sequences(train_captions)

In [0]:
# Rellena cada vector hasta la longitud máxima de los subtítulos
# Si no proporcionas un valor de max_length, pad_sequences lo calcula automáticamente
cap_vector = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')

In [0]:
# Calcula max_length, que se usa para almacenar los pesos de atención
max_length = calc_max_length(train_seqs)

## Dividimos los datos en training y testing

In [0]:
# Creamos los conjuntos de entrenamiento y validación usando una división 80-20
img_name_train, img_name_val, cap_train, cap_val = train_test_split(img_name_vector,
                                                                    cap_vector,
                                                                    test_size=0.2,
                                                                    random_state=0)

In [0]:
len(img_name_train), len(cap_train), len(img_name_val), len(cap_val)

## Creamos un dataset tf.data para el entrenamiento


¡Nuestras imágenes y subtítulos están listos! A continuación, creemos un conjunto de datos tf.data para usar y entrenar nuestro modelo.


In [0]:
# No dudes en cambiar estos parámetros de acuerdo con la configuración de tu sistema

BATCH_SIZE = 64
BUFFER_SIZE = 1000
embedding_dim = 256
units = 512
vocab_size = top_k + 1
num_steps = len(img_name_train) // BATCH_SIZE
# Shape of the vector extracted from InceptionV3 is (64, 2048)
# These two variables represent that vector shape
features_shape = 2048
attention_features_shape = 64

In [0]:
# Cargamos los archivos numpy
def map_func(img_name, cap):
  img_tensor = np.load(img_name.decode('utf-8')+'.npy')
  return img_tensor, cap

In [0]:
dataset = tf.data.Dataset.from_tensor_slices((img_name_train, cap_train))

# Usa el mapa para cargar los archivos numpy en paralelo
dataset = dataset.map(lambda item1, item2: tf.numpy_function(
          map_func, [item1, item2], [tf.float32, tf.int32]),
          num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Shuffle y batch
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

## Modelo

Dato curioso: el decodificador a continuación es idéntico al del ejemplo de [Neural Machine Translation with Attention](../sequences/nmt_with_attention.ipynb).

La arquitectura del modelo está inspirada en el artículo [Show, Attend and Tell](https://arxiv.org/pdf/1502.03044.pdf).

* En este ejemplo, extraemos las características de la capa convolucional inferior de Inception V3 dándonos un vector de forma (8, 8, 2048).
* Haremos un reshape a (64, 2048).
* Este vector se pasa a través del codificador CNN (que consiste en una sola capa totalmente conectada).
* El RNN (aquí GRU) atiende la imagen para predecir la siguiente palabra.

In [0]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, features, hidden):
    # características(CNN_encoder output) shape == (batch_size, 64, embedding_dim)

    # hidden shape == (batch_size, hidden_size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden_size)
    hidden_with_time_axis = tf.expand_dims(hidden, 1)

    # score shape == (batch_size, 64, hidden_size)
    score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))

    # attention_weights shape == (batch_size, 64, 1)
    # you get 1 at the last axis because you are applying score to self.V
    attention_weights = tf.nn.softmax(self.V(score), axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * features
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [0]:
class CNN_Encoder(tf.keras.Model):
    # Ya que ya se han extraido las características y las descargamos usando pickle
    # Este codificador pasa esas características a través de una capa totalmente conectada
    def __init__(self, embedding_dim):
        super(CNN_Encoder, self).__init__()
        # shape after fc == (batch_size, 64, embedding_dim)
        self.fc = tf.keras.layers.Dense(embedding_dim)

    def call(self, x):
        x = self.fc(x)
        x = tf.nn.relu(x)
        return x

In [0]:
class RNN_Decoder(tf.keras.Model):
  def __init__(self, embedding_dim, units, vocab_size):
    super(RNN_Decoder, self).__init__()
    self.units = units

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc1 = tf.keras.layers.Dense(self.units)
    self.fc2 = tf.keras.layers.Dense(vocab_size)

    self.attention = BahdanauAttention(self.units)

  def call(self, x, features, hidden):
    # definimos la atención como un modelo separado
    context_vector, attention_weights = self.attention(features, hidden)

    # x shape después de pasar por la incrustación == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape después de la concatenación == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # Pasamos el vector concatenado al GRU 
    output, state = self.gru(x)

    # shape == (batch_size, max_length, hidden_size)
    x = self.fc1(output)

    # x shape == (batch_size * max_length, hidden_size)
    x = tf.reshape(x, (-1, x.shape[2]))

    # output shape == (batch_size * max_length, vocab)
    x = self.fc2(x)

    return x, state, attention_weights

  def reset_state(self, batch_size):
    return tf.zeros((batch_size, self.units))

In [0]:
encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(embedding_dim, units, vocab_size)

In [0]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

## Checkpoint

In [0]:
checkpoint_path = "./checkpoints/train"
ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer = optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

In [0]:
start_epoch = 0
if ckpt_manager.latest_checkpoint:
  start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])

  # restauramos el último punto de control en checkpoint_path
  ckpt.restore(ckpt_manager.latest_checkpoint)

## Entrenamiento

* Extrae las características almacenadas en los respectivos archivos `.npy` y luego pasa esas características a través del codificador.
* La salida del codificador, el estado oculto (inicializado a 0) y la entrada del decodificador (que es el token de inicio) se pasan al decodificador.
* El decodificador devuelve las predicciones y el estado oculto del decodificador.
* El estado oculto del decodificador se devuelve al modelo y las predicciones se utilizan para calcular la pérdida.
* Usará un 'forzamiento maestro' para decidir la próxima entrada al decodificador.
* El forzamiento maestro es la técnica en la que se pasa la palabra objetivo como la siguiente entrada al decodificador.
* El paso final es calcular los gradientes y aplicarlos al optimizador y la retropropagación.


In [0]:
# agregamos esto en una celda separada porque si ejecuta la celda de entrenamiento
# muchas veces, la matriz loss_plot se restablecerá
loss_plot = []

In [0]:
@tf.function
def train_step(img_tensor, target):
  loss = 0

  # inicializando el estado oculto para cada batch
  # porque los subtítulos no están relacionados de una imagen a otra
  hidden = decoder.reset_state(batch_size=target.shape[0])

  dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * target.shape[0], 1)

  with tf.GradientTape() as tape:
      features = encoder(img_tensor)

      for i in range(1, target.shape[1]):
          # Pasamos las características a través del decodificador
          predictions, hidden, _ = decoder(dec_input, features, hidden)

          loss += loss_function(target[:, i], predictions)

          # usamos el teacher forcing ("forzamiento maestro")
          dec_input = tf.expand_dims(target[:, i], 1)

  total_loss = (loss / int(target.shape[1]))

  trainable_variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, trainable_variables)

  optimizer.apply_gradients(zip(gradients, trainable_variables))

  return loss, total_loss

In [0]:
EPOCHS = 20

for epoch in range(start_epoch, EPOCHS):
    start = time.time()
    total_loss = 0

    for (batch, (img_tensor, target)) in enumerate(dataset):
        batch_loss, t_loss = train_step(img_tensor, target)
        total_loss += t_loss

        if batch % 100 == 0:
            print ('Epoch {} Batch {} Loss {:.4f}'.format(
              epoch + 1, batch, batch_loss.numpy() / int(target.shape[1])))
    # almacenamos el valor de pérdida final de las épocas para trazarlo más tarde
    loss_plot.append(total_loss / num_steps)

    if epoch % 5 == 0:
      ckpt_manager.save()

    print ('Epoch {} Loss {:.6f}'.format(epoch + 1,
                                         total_loss/num_steps))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

In [0]:
plt.plot(loss_plot)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss Plot')
plt.show()

## ¡Subtítulos!

* La función de evaluación es similar al ciclo de entrenamiento, excepto que no utiliza el forzamiento maestro aquí. La entrada al decodificador en cada intervalo de tiempo son sus predicciones anteriores junto con el estado oculto y la salida del codificador.
* Deja de predecir cuando el modelo predice el token final.
* Y almacena los pesos de atención para cada intervalo de tiempo.

In [0]:
def evaluate(image):
    attention_plot = np.zeros((max_length, attention_features_shape))

    hidden = decoder.reset_state(batch_size=1)

    temp_input = tf.expand_dims(load_image(image)[0], 0)
    img_tensor_val = image_features_extract_model(temp_input)
    img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

    features = encoder(img_tensor_val)

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)
    result = []

    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(dec_input, features, hidden)

        attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()

        predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()
        result.append(tokenizer.index_word[predicted_id])

        if tokenizer.index_word[predicted_id] == '<end>':
            return result, attention_plot

        dec_input = tf.expand_dims([predicted_id], 0)

    attention_plot = attention_plot[:len(result), :]
    return result, attention_plot

In [0]:
def plot_attention(image, result, attention_plot):
    temp_image = np.array(Image.open(image))

    fig = plt.figure(figsize=(10, 10))

    len_result = len(result)
    for l in range(len_result):
        temp_att = np.resize(attention_plot[l], (8, 8))
        ax = fig.add_subplot(len_result//2, len_result//2, l+1)
        ax.set_title(result[l])
        img = ax.imshow(temp_image)
        ax.imshow(temp_att, cmap='gray', alpha=0.6, extent=img.get_extent())

    plt.tight_layout()
    plt.show()

In [0]:
# Subtítulos en el set de validación
rid = np.random.randint(0, len(img_name_val))
image = img_name_val[rid]
real_caption = ' '.join([tokenizer.index_word[i] for i in cap_val[rid] if i not in [0]])
result, attention_plot = evaluate(image)

print ('Real Caption:', real_caption)
print ('Prediction Caption:', ' '.join(result))
plot_attention(image, result, attention_plot)


## Inténtalo con tus propias imágenes

Por diversión, a continuación proporcionamos un método que puedes usar para subtitular tus propias imágenes con el modelo que acabamos de entrenar. Ten en cuenta que fue entrenado con una cantidad relativamente pequeña de datos, y tus imágenes pueden ser diferentes de los datos de entrenamiento (¡así que prepárate para obtener resultados extraños!)

In [0]:
image_url = 'https://tensorflow.org/images/surf.jpg'
image_extension = image_url[-4:]
image_path = tf.keras.utils.get_file('image'+image_extension,
                                     origin=image_url)

result, attention_plot = evaluate(image_path)
print ('Prediction Caption:', ' '.join(result))
plot_attention(image_path, result, attention_plot)
# Abrimos la imagen
Image.open(image_path)

# Siguientes pasos

¡Felicidades! Acabas de entrenar un modelo de subtítulos de imágenes con atención. A continuación, echa un vistazo a este ejemplo.
[Neural Machine Translation with Attention](../sequences/nmt_with_attention.ipynb). Utiliza una arquitectura similar para traducir oraciones en español e inglés. También puedes experimentar entrenando el código en este cuaderno en un conjunto de datos diferente.